In [ ]:
import sys
#edit below to include your directory as needed
#sys.path.append("/Users/blatham/Hacktoberfest21")
#sys.path.append("/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages")
import gmsh
import numpy as np

In [ ]:
#begin Model
gmsh.initialize()
gmsh.model.add('clover_mesh')

In [ ]:
#Parameters
h = 0.1
l = 0
L = 2
delta = 0.5
LL = L+ delta
Angle1 = np.pi/4
Angle2 = np.pi/6
C = np.cos(Angle1)
S = np.sin(Angle1)
Cp = np.cos(Angle1 + Angle2)
Cm = np.cos(Angle1 - Angle2)
Sp = np.sin(Angle1 + Angle2)
Sm = np.sin(Angle1 - Angle2)
Cph = np.cos(Angle1 + 0.5 * Angle2)
Cmh = np.cos(Angle1 - 0.5 * Angle2)
Sph = np.sin(Angle1 + 0.5 * Angle2)
Smh = np.sin(Angle1 - 0.5 * Angle2)
Rx = np.abs(L*Cp - 0.5*L*(Cp+0.5*(Cp+Cm)))
Ry = np.abs(L*Sp - 0.5*L*(Sp+0.5*(Sp+Sm)))
R = np.sqrt(Rx**2 + Ry**2)
InnerCircleRadius=0.3

In [ ]:
#Points
Points=[]
Points.append((l*Cp, l*Sp, 0, h))
Points.append((L*Cp, L*Sp, 0, h))
Points.append((0.5* L* (Cp+Cm), 0.5* L* (Sp+Sm), 0, h))
#The below point is commented out in the original mesh - not sure if it is to be included.
Points.append((l*Cm, l*Sm, 0, h))
Points.append((L*Cm, L*Sm, 0, h))
Points.append((0.5*L*(Cp+0.5*(Cp+Cm)), 0.5*L*(Sp+0.5*(Sp+Sm)), 0, h))
Points.append((0.5*L*(Cm+0.5*(Cp+Cm)), 0.5*L*(Sm+0.5*(Sp+Sm)), 0, h))
Points.append((0.5*L*(Cp+0.5*(Cp+Cm))+R*np.cos(np.pi/4), 0.5*L*(Sp+0.5*(Sp+Sm))+R*np.sin(np.pi/4), 0, h))
Points.append((0.5*L*(Cm+0.5*(Cp+Cm))+R*np.cos(np.pi/4), 0.5*L*(Sm+0.5*(Sp+Sm))+R*np.sin(np.pi/4), 0, h))

Points.append((InnerCircleRadius*Cp,InnerCircleRadius*Sp,0,h))
Points.append((InnerCircleRadius*Cm,InnerCircleRadius*Sm,0,h))
Points.append((InnerCircleRadius,0,0,h))
Points.append((0,InnerCircleRadius,0,h))
#Points.append((InnerCircleRadius*np.cos(np.pi/4),InnerCircleRadius*(np.cos(np.pi/4)),0,h))
geoPoints= [gmsh.model.geo.addPoint(*p) for p in Points]

In [ ]:
geoPoints

In [ ]:
#Lines
Lines=[]
Lines.append((10,2))
Lines.append((11,5))
#Lines.append((1,10))
#Lines.append((1,11))
Lines.append((1,12))
Lines.append((1,13))
geoLines=[gmsh.model.geo.addLine(*L) for L in Lines]

In [ ]:
#Circles
Circles=[]
Circles.append((2,6,8))
Circles.append((8,6,3))
Circles.append((3,7,9))
Circles.append((9,7,5))
Circles.append((12,1,11))
Circles.append((10,1,13))

geoCircles=[gmsh.model.geo.addCircleArc(*C) for C in Circles]

In [ ]:
#loop and surface
gmsh.model.geo.addCurveLoops((3,9,1,5,6,7,8,-2,10,-4))

In [ ]:
Surfaces=[]
gmsh.model.geo.addPlaneSurface([1])

In [ ]:
Surfaces.append([(2,1)])
Surfaces.append(gmsh.model.geo.copy([(2,1)]))
gmsh.model.geo.symmetrize(Surfaces[1],1,0,0,0)
Surfaces.append(gmsh.model.geo.copy([(2,1)]))
gmsh.model.geo.symmetrize(Surfaces[2],0,1,0,0)
Surfaces.append(gmsh.model.geo.copy([(2,1)]))
gmsh.model.geo.symmetrize(Surfaces[3],1,1,0,0)

In [ ]:
gmsh.model.geo.addSurfaceLoop((1,11,21,32))

In [ ]:
Interface=gmsh.model.geo.addPhysicalGroup(1,(9,2,8,7,6,5,-1,10,20,19,18,17,16,15,-14,13,41,40,39,38,37,36,-35,34,30,29,28,27,26,25,-24,23))
gmsh.model.setPhysicalName(1,Interface,'Interface')

In [ ]:
clover=gmsh.model.geo.addPhysicalGroup(2,(1,11,21,32))
gmsh.model.setPhysicalName(2,clover,'clover')

In [ ]:
externalpts=[]
externalpts.append((3, 0, 0, h))
externalpts.append((0, 3, 0, h))
externalpts.append((-3, 0, 0, h))
externalpts.append((0, -3, 0, h))
geoexPoints= [gmsh.model.geo.addPoint(*p) for p in externalpts]

In [ ]:
geoexPoints

In [ ]:
outercircle=[]
outercircle.append((geoexPoints[0],1,geoexPoints[1]))
outercircle.append((geoexPoints[1],1,geoexPoints[2]))
outercircle.append((geoexPoints[2],1,geoexPoints[3]))
outercircle.append((geoexPoints[3],1,geoexPoints[0]))
geoouterCircles=[gmsh.model.geo.addCircleArc(*C) for C in outercircle]

In [ ]:
geoouterCircles

In [ ]:
gmsh.model.geo.addCurveLoops(geoouterCircles)

In [ ]:
gmsh.model.geo.addCurveLoops((9,2,8,7,6,5,-1,10,20,19,18,17,16,15,-14,13,41,40,39,38,37,36,-35,34,30,29,28,27,26,25,-24,23))

In [ ]:
gmsh.model.geo.addPlaneSurface([2,3])

In [ ]:
Boundary=gmsh.model.geo.addPhysicalGroup(1,geoouterCircles)
gmsh.model.setPhysicalName(1,Boundary,'Boundary')

In [ ]:
Vacuum=gmsh.model.geo.addPhysicalGroup(2,[33])
gmsh.model.setPhysicalName(2,Vacuum,'Vacuum')

In [ ]:
gmsh.model.geo.synchronize()

In [ ]:
#meshing
gmsh.model.mesh.generate(2)
gmsh.option.setNumber("Mesh.MshFileVersion",2.2)

gmsh.write('clover_mesh.msh')

In [ ]:
#optional gui cell to see mesh, uncomment if wanted
#gmsh.fltk.run()

In [ ]:
#closing

gmsh.finalize()